In [ ]:
%cd drive/My Drive/Estudos/titanic-train

/content/drive/My Drive/Estudos/titanic-train


In [ ]:
import pandas as pd
import glob

### Lendo o arquivo com os resultados de validação




In [ ]:
resultados_validacao = pd.read_csv('resultados_treino.csv', sep=';')

resultados_validacao.head()

,tecnica,accuracy,roc_auc_score,auc,f1_score,log_loss,precision,recall,identificador_aleatorio1,identificador_aleatorio2,tipo_split,normalizador,redutor_dimensionalidade,pipeline,parametros,random_state_split,random_state_random_search,cross_validation_random_search,best_estimator,best_params,versao_dados
0,AdaBoostClassifier,0.791444,0.794082,0.884118,0.748387,7.203381,0.698795,0.805556,5107697,8380792,train_test_split 30%,robust_scaler,nmf,"[('robust_scaler', RobustScaler(copy=True, qua...","{'robust_scaler__with_centering': [True, False...",29,41,8,"Pipeline(memory=None,\n steps=[('robus...","{'robust_scaler__with_scaling': False, 'robust...",3_4
1,BaggingClassifier,0.796791,0.754287,0.872041,0.683333,7.018605,0.854167,0.569444,9448123,3718929,train_test_split 30%,robust_scaler,nmf,"[('robust_scaler', RobustScaler(copy=True, qua...","{'robust_scaler__with_centering': [True, False...",29,15,9,"Pipeline(memory=None,\n steps=[('robus...","{'robust_scaler__with_scaling': False, 'robust...",3_4
2,BernoulliNB,0.764706,0.759360,0.836413,0.706667,8.126878,0.679487,0.736111,1208291,7959870,train_test_split 30%,robust_scaler,nmf,"[('robust_scaler', RobustScaler(copy=True, qua...","{'robust_scaler__with_centering': [True, False...",29,27,3,"Pipeline(memory=None,\n steps=[('robus...","{'robust_scaler__with_scaling': True, 'robust_...",3_4
3,CalibratedClassifierCV,0.780749,0.785386,0.876027,0.738854,7.572788,0.682353,0.805556,8341088,8219626,train_test_split 30%,robust_scaler,nmf,"[('robust_scaler', RobustScaler(copy=True, qua...","{'robust_scaler__with_centering': [True, False...",29,7,7,"Pipeline(memory=None,\n steps=[('robus...","{'robust_scaler__with_scaling': False, 'robust...",3_4
4,ExtraTreeClassifier,0.577540,0.633152,0.823732,0.614634,14.591547,0.473684,0.875000,1942461,1698129,train_test_split 30%,robust_scaler,nmf,"[('robust_scaler', RobustScaler(copy=True, qua...","{'robust_scaler__with_centering': [True, False...",29,24,5,"Pipeline(memory=None,\n steps=[('robus...","{'robust_scaler__with_scaling': False, 'robust...",3_4


##### Identificando os 15 melhores desempenhos com base na métrica 'auc'

In [ ]:
resultados_validacao.sort_values('auc', ascending=False).head(15)

,tecnica,accuracy,roc_auc_score,auc,f1_score,log_loss,precision,recall,identificador_aleatorio1,identificador_aleatorio2,tipo_split,normalizador,redutor_dimensionalidade,pipeline,parametros,random_state_split,random_state_random_search,cross_validation_random_search,best_estimator,best_params,versao_dados
382,LabelSpreading,0.848,0.820721,0.921024,0.776471,5.249932,0.846154,0.717391,4381312,3915286,train_test_split 20%,min_max_scaler,nmf,"[('min_max_scaler', MinMaxScaler(copy=True, fe...","{'min_max_scaler__feature_range': [(0, 1), (1,...",18,35,3,"Pipeline(memory=None,\n steps=[('min_m...","{'nmf__solver': 'mu', 'nmf__n_components': 3, ...",3_4
104,GradientBoostingClassifier,0.616,0.500000,0.916396,0.000000,13.262890,0.000000,0.000000,6767531,7611493,train_test_split 20%,Não utilizado,Não utilizado,"[('GradientBoostingClassifier', GradientBoosti...",{'GradientBoostingClassifier__loss': ('devianc...,25,18,6,"Pipeline(memory=None,\n steps=[('Gradi...",{'GradientBoostingClassifier__max_features': '...,3_4
385,LogisticRegressionCV,0.816,0.768162,0.915933,0.701299,6.355160,0.870968,0.586957,9523798,7686607,train_test_split 20%,min_max_scaler,nmf,"[('min_max_scaler', MinMaxScaler(copy=True, fe...","{'min_max_scaler__feature_range': [(0, 1), (1,...",18,10,6,"Pipeline(memory=None,\n steps=[('min_m...","{'nmf__solver': 'mu', 'nmf__n_components': Non...",3_4
383,LinearDiscriminantAnalysis,0.848,0.829802,0.910704,0.786517,5.249945,0.813953,0.760870,9966208,1659430,train_test_split 20%,min_max_scaler,nmf,"[('min_max_scaler', MinMaxScaler(copy=True, fe...","{'min_max_scaler__feature_range': [(0, 1), (1,...",18,35,8,"Pipeline(memory=None,\n steps=[('min_m...","{'nmf__solver': 'cd', 'nmf__n_components': Non...",3_4
97,BaggingClassifier,0.848,0.821699,0.909903,0.781609,5.249926,0.871795,0.708333,471614,637850,train_test_split 20%,Não utilizado,Não utilizado,"[('BaggingClassifier', BaggingClassifier(base_...","{'BaggingClassifier__n_estimators': [5, 10, 15...",25,2,3,"Pipeline(memory=None,\n steps=[('Baggi...","{'BaggingClassifier__oob_score': True, 'Baggin...",3_4
378,GaussianProcessClassifier,0.856,0.845212,0.909604,0.804348,4.973641,0.804348,0.804348,9573760,1102412,train_test_split 20%,min_max_scaler,nmf,"[('min_max_scaler', MinMaxScaler(copy=True, fe...","{'min_max_scaler__feature_range': [(0, 1), (1,...",18,7,4,"Pipeline(memory=None,\n steps=[('min_m...","{'nmf__solver': 'mu', 'nmf__n_components': 7, ...",3_4
386,MLPClassifier,0.856,0.845212,0.909329,0.804348,4.973641,0.804348,0.804348,7509109,8174908,train_test_split 20%,min_max_scaler,nmf,"[('min_max_scaler', MinMaxScaler(copy=True, fe...","{'min_max_scaler__feature_range': [(0, 1), (1,...",18,1,3,"Pipeline(memory=None,\n steps=[('min_m...","{'nmf__solver': 'cd', 'nmf__n_components': 7, ...",3_4
439,MLPClassifier,0.800,0.797463,0.905074,0.747475,6.907851,0.711538,0.787234,4212105,9533835,train_test_split 20%,robust_scaler,nmf,"[('robust_scaler', RobustScaler(copy=True, qua...","{'robust_scaler__with_centering': [True, False...",41,35,6,"Pipeline(memory=None,\n steps=[('robus...","{'robust_scaler__with_scaling': False, 'robust...",3_4
735,MLPClassifier,0.824,0.820922,0.905074,0.775510,6.078908,0.745098,0.808511,1757616,8390112,train_test_split 20%,robust_scaler,Não utilizado,"[('robust_scaler', RobustScaler(copy=True, qua...","{'robust_scaler__with_centering': [True, False...",41,4,9,"Pipeline(memory=None,\n steps=[('robus...","{'robust_scaler__with_scaling': False, 'robust...",3_4
371,AdaBoostClassifier,0.856,0.845212,0.905063,0.804348,4.973641,0.804348,0.804348,5686296,7827836,train_test_split 20%,min_max_scaler,nmf,"[('min_max_scaler', MinMaxScaler(copy=True, fe...","{'min_max_scaler__feature_range': [(0, 1), (1,...",18,28,10,"Pipeline(memory=None,\n steps=[('min_m...","{'nmf__solver': 'mu', 'nmf__n_components': 7, ...",3_4


In [ ]:
# Accuracy
resultados_validacao.sort_values('accuracy', ascending=False).head(15)

,tecnica,accuracy,roc_auc_score,auc,f1_score,log_loss,precision,recall,identificador_aleatorio1,identificador_aleatorio2,tipo_split,normalizador,redutor_dimensionalidade,pipeline,parametros,random_state_split,random_state_random_search,cross_validation_random_search,best_estimator,best_params,versao_dados
725,DecisionTreeClassifier,0.888,0.876432,0.870704,0.847826,3.868381,0.866667,0.829787,6523596,4531328,train_test_split 20%,robust_scaler,Não utilizado,"[('robust_scaler', RobustScaler(copy=True, qua...","{'robust_scaler__with_centering': [True, False...",41,35,7,"Pipeline(memory=None,\n steps=[('robus...","{'robust_scaler__with_scaling': True, 'robust_...",3_4
101,ExtraTreeClassifier,0.888,0.873782,0.872835,0.847826,3.868375,0.886364,0.812500,596494,4600321,train_test_split 20%,Não utilizado,Não utilizado,"[('ExtraTreeClassifier', ExtraTreeClassifier(c...","{'ExtraTreeClassifier__criterion': ('gini', 'e...",25,31,5,"Pipeline(memory=None,\n steps=[('Extra...","{'ExtraTreeClassifier__max_features': 'auto', ...",3_4
722,BaggingClassifier,0.880,0.870022,0.873432,0.838710,4.144698,0.847826,0.829787,7734583,317123,train_test_split 20%,robust_scaler,Não utilizado,"[('robust_scaler', RobustScaler(copy=True, qua...","{'robust_scaler__with_centering': [True, False...",41,27,10,"Pipeline(memory=None,\n steps=[('robus...","{'robust_scaler__with_scaling': True, 'robust_...",3_4
737,RandomForestClassifier,0.872,0.863612,0.889662,0.829787,4.421015,0.829787,0.829787,3622886,8562784,train_test_split 20%,robust_scaler,Não utilizado,"[('robust_scaler', RobustScaler(copy=True, qua...","{'robust_scaler__with_centering': [True, False...",41,3,10,"Pipeline(memory=None,\n steps=[('robus...","{'robust_scaler__with_scaling': True, 'robust_...",3_4
380,KNeighborsClassifier,0.872,0.844249,0.900110,0.809524,4.420989,0.894737,0.739130,1678358,967420,train_test_split 20%,min_max_scaler,nmf,"[('min_max_scaler', MinMaxScaler(copy=True, fe...","{'min_max_scaler__feature_range': [(0, 1), (1,...",18,24,10,"Pipeline(memory=None,\n steps=[('min_m...","{'nmf__solver': 'mu', 'nmf__n_components': 3, ...",3_4
428,DecisionTreeClassifier,0.864,0.852973,0.843835,0.817204,4.697325,0.826087,0.808511,7542554,1865680,train_test_split 20%,robust_scaler,nmf,"[('robust_scaler', RobustScaler(copy=True, qua...","{'robust_scaler__with_centering': [True, False...",41,37,10,"Pipeline(memory=None,\n steps=[('robus...","{'robust_scaler__with_scaling': True, 'robust_...",3_4
102,ExtraTreesClassifier,0.864,0.850379,0.901650,0.817204,4.697318,0.844444,0.791667,9005715,8292095,train_test_split 20%,Não utilizado,Não utilizado,"[('ExtraTreesClassifier', ExtraTreesClassifier...","{'ExtraTreesClassifier__criterion': ('gini', '...",25,42,8,"Pipeline(memory=None,\n steps=[('Extra...","{'ExtraTreesClassifier__max_features': 'sqrt',...",3_4
726,ExtraTreeClassifier,0.864,0.844517,0.821331,0.808989,4.697312,0.857143,0.765957,4852855,1599411,train_test_split 20%,robust_scaler,Não utilizado,"[('robust_scaler', RobustScaler(copy=True, qua...","{'robust_scaler__with_centering': [True, False...",41,13,8,"Pipeline(memory=None,\n steps=[('robus...","{'robust_scaler__with_scaling': False, 'robust...",3_4
100,DecisionTreeClassifier,0.864,0.846456,0.851326,0.813187,4.697312,0.860465,0.770833,2173780,3694341,train_test_split 20%,Não utilizado,Não utilizado,"[('DecisionTreeClassifier', DecisionTreeClassi...","{'DecisionTreeClassifier__criterion': ('gini',...",25,4,5,"Pipeline(memory=None,\n steps=[('Decis...","{'DecisionTreeClassifier__splitter': 'best', '...",3_4
378,GaussianProcessClassifier,0.856,0.845212,0.909604,0.804348,4.973641,0.804348,0.804348,9573760,1102412,train_test_split 20%,min_max_scaler,nmf,"[('min_max_scaler', MinMaxScaler(copy=True, fe...","{'min_max_scaler__feature_range': [(0, 1), (1,...",18,7,4,"Pipeline(memory=None,\n steps=[('min_m...","{'nmf__solver': 'mu', 'nmf__n_components': 7, ...",3_4


***

# Retreinar melhor modelo com os dados completos, sem split

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import FastICA
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA, NMF
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.linear_model import PassiveAggressiveClassifier, LogisticRegressionCV
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.semi_supervised import LabelSpreading
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import joblib

In [ ]:
dados_train = pd.read_csv('dados/train_tratado-3_4.csv', sep=';')
dados_train.head()

,Survived,Fare,Pclass_3,Sex_male,Name_Miss,Name_Mr,Name_Mrs,Cabin_B,Cabin_D,Cabin_N,PassengerId
0,0.0,52.0000,0,1,0,1,0,0,0,0,476
1,1.0,10.5000,0,0,1,0,0,0,0,1,57
2,1.0,512.3292,0,0,1,0,0,0,0,1,259
3,1.0,30.0000,0,1,0,1,0,0,1,0,741
4,0.0,110.8833,0,1,0,1,0,0,0,0,699


In [ ]:
X_train = dados_train.drop(columns=['PassengerId', 'Survived'])
y_train = dados_train['Survived']
X_train.head()

,Fare,Pclass_3,Sex_male,Name_Miss,Name_Mr,Name_Mrs,Cabin_B,Cabin_D,Cabin_N
0,52.0000,0,1,0,1,0,0,0,0
1,10.5000,0,0,1,0,0,0,0,1
2,512.3292,0,0,1,0,0,0,0,1
3,30.0000,0,1,0,1,0,0,1,0
4,110.8833,0,1,0,1,0,0,0,0


In [ ]:
#melhor_modelo = resultados_validacao.sort_values('roc_auc_score', ascending=False).head(1)
melhor_modelo = resultados_validacao.loc[(resultados_validacao['identificador_aleatorio1'] == 9573760) & (resultados_validacao['identificador_aleatorio2'] == 1102412)]
melhor_modelo

,tecnica,accuracy,roc_auc_score,auc,f1_score,log_loss,precision,recall,identificador_aleatorio1,identificador_aleatorio2,tipo_split,normalizador,redutor_dimensionalidade,pipeline,parametros,random_state_split,random_state_random_search,cross_validation_random_search,best_estimator,best_params,versao_dados
378,GaussianProcessClassifier,0.856,0.845212,0.909604,0.804348,4.973641,0.804348,0.804348,9573760,1102412,train_test_split 20%,min_max_scaler,nmf,"[('min_max_scaler', MinMaxScaler(copy=True, fe...","{'min_max_scaler__feature_range': [(0, 1), (1,...",18,7,4,"Pipeline(memory=None,\n steps=[('min_m...","{'nmf__solver': 'mu', 'nmf__n_components': 7, ...",3_4


In [ ]:
parametros_pipeline = eval(melhor_modelo['pipeline'].values[0].replace(' ', '').replace('\n', ''))
parametros_pipeline

[('min_max_scaler', MinMaxScaler(copy=True, feature_range=(0, 1))),
 ('nmf',
  NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
      n_components=None, random_state=None, shuffle=False, solver='cd',
      tol=0.0001, verbose=0)),
 ('GaussianProcessClassifier',
  GaussianProcessClassifier(copy_X_train=True, kernel=None, max_iter_predict=100,
                            multi_class='one_vs_rest', n_jobs=None,
                            n_restarts_optimizer=0, optimizer='fmin_l_bfgs_b',
                            random_state=None, warm_start=False))]

In [ ]:
parametros_pipeline_best = eval(melhor_modelo['best_estimator'].values[0].replace(' ', '').replace('\n', ''))
parametros_pipeline_best

Pipeline(memory=None,
         steps=[('min_max_scaler',
                 MinMaxScaler(copy=True, feature_range=(1, 10))),
                ('nmf',
                 NMF(alpha=0.0, beta_loss='frobenius', init='nndsvda',
                     l1_ratio=0.0, max_iter=200, n_components=7,
                     random_state=None, shuffle=False, solver='mu', tol=0.0001,
                     verbose=0)),
                ('GaussianProcessClassifier',
                 GaussianProcessClassifier(copy_X_train=True, kernel=None,
                                           max_iter_predict=1000,
                                           multi_class='one_vs_rest',
                                           n_jobs=None, n_restarts_optimizer=5,
                                           optimizer='fmin_l_bfgs_b',
                                           random_state=None,
                                           warm_start=False))],
         verbose=False)

In [ ]:
parametros_pipeline_best.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/decomposition/_nmf.py:1076: ConvergenceWarning: Maximum number of iteration 200 reached. Increase it to improve convergence.
  " improve convergence." % max_iter, ConvergenceWarning)


Pipeline(memory=None,
         steps=[('min_max_scaler',
                 MinMaxScaler(copy=True, feature_range=(1, 10))),
                ('nmf',
                 NMF(alpha=0.0, beta_loss='frobenius', init='nndsvda',
                     l1_ratio=0.0, max_iter=200, n_components=7,
                     random_state=None, shuffle=False, solver='mu', tol=0.0001,
                     verbose=0)),
                ('GaussianProcessClassifier',
                 GaussianProcessClassifier(copy_X_train=True, kernel=None,
                                           max_iter_predict=1000,
                                           multi_class='one_vs_rest',
                                           n_jobs=None, n_restarts_optimizer=5,
                                           optimizer='fmin_l_bfgs_b',
                                           random_state=None,
                                           warm_start=False))],
         verbose=False)

In [ ]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('min_max_scaler',
                 MinMaxScaler(copy=True, feature_range=(0, 1))),
                ('nmf',
                 NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0,
                     max_iter=200, n_components=None, random_state=None,
                     shuffle=False, solver='cd', tol=0.0001, verbose=0)),
                ('GaussianProcessClassifier',
                 GaussianProcessClassifier(copy_X_train=True, kernel=None,
                                           max_iter_predict=100,
                                           multi_class='one_vs_rest',
                                           n_jobs=None, n_restarts_optimizer=0,
                                           optimizer='fmin_l_bfgs_b',
                                           random_state=None,
                                           warm_start=False))],
         verbose=False)

In [ ]:
#nome_arquivo = 'modelos/melhor/best_9573760_1102412_GaussianProcessClassifier.pkl'
#joblib.dump(parst, nome_arquivo)

['modelos/melhor/best_9573760_1102412_GaussianProcessClassifier.pkl']